# San Francisco Police Department Incident Dataset

In this notebook, we'll be exploring the incident report dataset from the San Francisco Police Department using data mining and visualization techniques.

----

## Library Code

In order to make code development easier and to not clutter the notebook,
we'll be referring to definitions from our own notebook-local Python library
in the following sections.

In [ ]:
import crime

----

## Reading the Dataset

In [ ]:
import pandas

Our raw source dataset can be retrieved from this link: <https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD>.

It is larger than the maximum 100 MB file size that github allows.  Until we decide how we're going to filter it and can check it in, just make certain it's downloaded into the same directory as the ipython notebook file.

In [ ]:
in_file = 'SFPD_Incidents_-_from_1_January_2003.csv'

### The Big Picture

Once the dataset is in place, the following code can read in the CSV as a dataframe and report the number of rows.

In [ ]:
in_df = pandas.read_csv(in_file)
print("Read {} rows x {} columns of input data from '{}'.".format(in_df.shape[0], in_df.shape[1], in_file))
in_df.head(50)

Print the list of features from the original dataset so that we can see what kinds of columns we're working with.

In [ ]:
in_features = list(in_df.columns.values)
print(in_features)

Let's get a little bit more information about the types of values in our input data.

In [ ]:
in_df.info()

In [ ]:
in_df.describe()

In [ ]:
in_df.describe(include=['O'])

### Zooming In

It looks to me like IncidntNum has identical values in one to several adjacent rows through the data set.  Looking further at the details of such groupings of rows, it appears that this column is used as a key to represent several citations, each with unique Category and Descript, for a single incident that happened at the same Date/Time/Location/etc.

Given this, it would be interesting to know how many unique values are in this column in order to compare to the total number of rows.

In [ ]:
incidntnum_values = list(in_df.groupby('IncidntNum').IncidntNum.nunique().index.values)
print("The 'IncidntNum' column contains {} unique values.".format(len(incidntnum_values)))

Next, it is likely to prove useful to know both the count and the actual values for the unique values within each of the Resolution, Category, and Descript columns.

In [ ]:
category_values = list(in_df.groupby('Category').Category.nunique().index.values)
print("The 'Category' column contains {} unique values.".format(len(category_values)))
print(category_values)

In [ ]:
descript_values = list(in_df.groupby('Descript').Descript.nunique().index.values)
print("The 'Descript' column contains {} unique values.".format(len(descript_values)))
print(descript_values)

In [ ]:
resolution_values = list(in_df.groupby('Resolution').Resolution.nunique().index.values)
print("The 'Resolution' column contains {} unique values.".format(len(resolution_values)))
print(resolution_values)

----

## Cleaning the Dataset

Some of the columns in the input dataset need to be cleaned up before we can make much use of them.

In [ ]:
incident_df = in_df

To begin with, observe that the 'Date' and 'Time' strings should be converted to a numeric Timestamp.

In [ ]:
import datetime
#incident_df['Date'] = pandas.to_datetime(incident_df['Date'])
date = incident_df.get_value(0, 'Date')
pandas.to_datetime(date)

----

## Reducing the Dataset

Examining the available features, we can see several columns that are of little interest to the rest of the work.  So, let's immediately filter for the interesting features.

In [ ]:
filtered_df = incident_df.filter(items=['Category', 'Descript', 'Date', 'Time', 'Resolution', 'X', 'Y'])
print("Filtered incident data down to {} rows x {} columns.".format(filtered_df.shape[0], filtered_df.shape[1]))

Now that we've reduced the dimensionality of our dataset by eliminating uninteresting columns, we can also consider selecting fewer rows.  Looking at the classes of 'Resolution' values, it might make sense to constrain our work over the set of instances that resulted in an arrest and booking and then eliminate the 'Resolution' column.

This is a neat way to perform a select to filter rows on a dataframe. Be careful, though, and note that the row indices still match those from the original dataset, which will cause you fits if you're trying to iterate over row indices with the smaller dataset since it no longer has contiguous indices.

In [ ]:
bookings_df = filtered_df[filtered_df.Resolution == 'ARREST, BOOKED']
bookings_df = bookings_df.drop('Resolution', axis=1)
print("Bookings data reduced down to {} rows x {} columns.".format(bookings_df.shape[0], bookings_df.shape[1]))

Let's take a look at the shape of the reduced bookings dataset.

In [ ]:
bookings_df.info()

In [ ]:
bookings_df.describe()

In [ ]:
bookings_df.describe(include=['O'])

NOTE: For whatever it's worth, I'm starting to second guess filtering the incidents down to just bookings.

----

## Initializing Bokeh

Bokeh is a useful Python visualization library that we'll be using multiple times below.  Rather than repeat its initialization or spread it out over multiple locations in the document, I'm giving its initialization an independent section early in the notebook.

Also, please note that I had to run the following command before I could get Bokeh plots to display at all.

```
jupyter nbextension enable --py --sys-prefix widgetsnbextension
```

In [ ]:
# Initialize Bokeh for visualizations.
import bokeh.io
import bokeh.models
import bokeh.plotting

bokeh.io.output_notebook()

----

## Mapping Incidents

It would be useful to be able to do visualizations involving
layering over a base map of San Francisco based upon coordinates
expressed like we have available in the dataset.

That said, our bookings dataframe still has several hundred thousand
incidents in it, so let's constrain our example visualization down
to a single explicit date, using whatever the date of the first
incident in our original input data was for arbitrary simplicity.

In [ ]:
date = incident_df.get_value(0, 'Date')
print(date)

In [ ]:
df_bookings_on_date = bookings_df[bookings_df.Date == date]
df_bookings_on_date = df_bookings_on_date.filter(items=['Category', 'Descript', 'Time', 'X', 'Y'])
df_bookings_on_date.head(5)

To start with for map visualization, we've been using Bokeh:
<http://bokeh.pydata.org/en/latest/docs/user_guide/geo.html>.

In [ ]:
bokeh.io.show(crime.map_incidents(df_bookings_on_date))

----

## Gaussian Naive Bayes

TODO: This is just chicken scratch so far.  I should probably move it to a branch for now.

NOTE: Before editing, the following snippet originally came from <https://www.kaggle.com/wikaiqi/titanic/titaniclearningqi>.

In [ ]:
## Gaussian Naive Bayes
#gaussian = GaussianNB()
#gaussian.fit(X_data, Y_data)
#Y_pred = gaussian.predict(X_test_kaggle)
#acc_gaussian = cross_val_score(gaussian, X_data, Y_data, cv=Kfold)
#bcc_gaussian = round(gaussian.score(X_test, Y_test) * 100, 5)

#submission = pd.DataFrame({
#        "PassengerId": test_df["PassengerId"],
#        "Survived": Y_pred
#    })
#submission.to_csv('submission_Gassian_Naive_Bayes.csv', index=False)

----

## Notes

*interesting things to experiment with:*

* See if clustering would show anything interesting in our dataset.
* Change the map to display different categories of incident in different colors.
* Write a getDayOfWeekFromDate() helper function in crime library.
* Linearize 'Date' and 'Time' fields into a single date/time value (e.g. unix time in seconds since 1970).